# Take home assignment - Front
## Ignacio Loyola Hernández

### Requirements

In [2]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

### Settings

In [12]:
API_BASE_URL = "https://api.weather.gov/stations"
STATION_ID = "0128W"  # Puedes cambiar este ID por el que prefieras

In [64]:
# get dict of stations names and timezones
def get_stations():
    response = requests.get(f"{API_BASE_URL}")
    stations = response.json().get('features', [])
    name_map = {station['properties']['stationIdentifier']: station['properties']['name'] for station in stations}
    timezone_map = {station['properties']['stationIdentifier']: station['properties']['timeZone'] for station in stations}
    return name_map, timezone_map

name_map, timezone_map = get_stations()[0], get_stations()[1]

In [26]:
# get weather information for a single station
def get_station_observations(station_id):
    url = f"{API_BASE_URL}/{station_id}/observations"
    response = requests.get(url)
    observations = response.json().get('features', [])
    return observations

In [33]:
observations = get_station_observations(STATION_ID)
observations[0]['properties']

{'@id': 'https://api.weather.gov/stations/0128W/observations/2024-09-04T16:40:00+00:00',
 '@type': 'wx:ObservationStation',
 'elevation': {'unitCode': 'wmoUnit:m', 'value': 11},
 'station': 'https://api.weather.gov/stations/0128W',
 'timestamp': '2024-09-04T16:40:00+00:00',
 'rawMessage': '',
 'textDescription': '',
 'icon': None,
 'presentWeather': [],
 'temperature': {'unitCode': 'wmoUnit:degC',
  'value': 33.72,
  'qualityControl': 'V'},
 'dewpoint': {'unitCode': 'wmoUnit:degC',
  'value': 26.97,
  'qualityControl': 'V'},
 'windDirection': {'unitCode': 'wmoUnit:degree_(angle)',
  'value': 165,
  'qualityControl': 'V'},
 'windSpeed': {'unitCode': 'wmoUnit:km_h-1',
  'value': 1.62,
  'qualityControl': 'V'},
 'windGust': {'unitCode': 'wmoUnit:km_h-1',
  'value': None,
  'qualityControl': 'Z'},
 'barometricPressure': {'unitCode': 'wmoUnit:Pa',
  'value': 101693.2,
  'qualityControl': 'V'},
 'seaLevelPressure': {'unitCode': 'wmoUnit:Pa',
  'value': None,
  'qualityControl': 'Z'},
 'visib

In [65]:
# Transform and create data points
def transform_observations(observations, name_map, timezone_map):
    transformed_data = []
    name = name_map.get(STATION_ID, "Unknown")  # Get station's timezone
    timezone = timezone_map.get(STATION_ID, "Unknown")  # Get station's timezone

    for obs in observations:
        # Verificar y extraer cada atributo asegurándose de que no sea None antes de aplicar round
        temp = obs['properties'].get('temperature')
        temperature = round(temp['value'], 2) if temp and temp.get('value') is not None else None
        
        wind_speed_data = obs['properties'].get('windSpeed')
        wind_speed = round(wind_speed_data['value'], 2) if wind_speed_data and wind_speed_data.get('value') is not None else None
        
        humidity_data = obs['properties'].get('relativeHumidity')
        humidity = round(humidity_data['value'], 2) if humidity_data and humidity_data.get('value') is not None else None
        
        data = {
            "station_id": STATION_ID,
            "station_name": name,
            "station_timezone": timezone,
            "latitude": obs['geometry']['coordinates'][1],
            "longitude": obs['geometry']['coordinates'][0],
            "timestamp": obs['properties']['timestamp'],
            "temperature": temperature,
            "wind_speed": wind_speed,
            "humidity": humidity
        }
        transformed_data.append(data)
    return transformed_data

In [66]:
obs = transform_observations(observations, name_map, timezone_map)

In [67]:
obs[0]

{'station_id': '0128W',
 'station_name': 'Ringling Museum of Art',
 'station_timezone': 'America/New_York',
 'latitude': 27.38,
 'longitude': -82.56,
 'timestamp': '2024-09-04T16:40:00+00:00',
 'temperature': 33.72,
 'wind_speed': 1.62,
 'humidity': 67.91}

In [68]:
df = pd.DataFrame(obs)

In [70]:
df.head()

,station_id,station_name,station_timezone,latitude,longitude,timestamp,temperature,wind_speed,humidity
0,0128W,Ringling Museum of Art,America/New_York,27.38,-82.56,2024-09-04T16:40:00+00:00,33.72,1.62,67.91
1,0128W,Ringling Museum of Art,America/New_York,27.38,-82.56,2024-09-04T16:30:00+00:00,33.00,4.82,69.23
2,0128W,Ringling Museum of Art,America/New_York,27.38,-82.56,2024-09-04T16:10:00+00:00,33.50,14.47,68.63
3,0128W,Ringling Museum of Art,America/New_York,27.38,-82.56,2024-09-04T15:50:00+00:00,32.78,12.89,71.63
4,0128W,Ringling Museum of Art,America/New_York,27.38,-82.56,2024-09-04T15:40:00+00:00,32.50,9.65,72.35
